## Example NaKL Model

In [5]:
# imports:
import os
import spyks.core as spk
from spyks import build

To define the model, we need to specify the state variables, how they change in time (the equations of motion), and any forcing terms and parameters. The string below contains these definition in YAML format. Normally we'd put this descriptor in a `.yml` file, but we can also compile it on the fly.

In [2]:
# define the model:
doc = """
---
name: nakl
description: biophysical neuron model with minimal Na, K, leak conductances
author: dmeliza
version: 1.0
state:
  V: -70 mV
  m: 0
  h: 0
  n: 0
forcing:
  Iinj: 0 pA
equations:
  V: 1/C * ((gna*m*m*m*h*(Ena - V)) + (gk*n*n*n*n*(Ek - V)) + (gl*(El-V)) + Iinj)
  m:
    inf: (1+tanh((V - vm)/dvm))/2
    tau: (tm0 + tm1 * (1 - tanh((V - vmt)/dvmt)**2))
  h:
    inf: (1+tanh((V - vh)/dvh))/2
    tau: (th0 + th1 * (1 - tanh((V - vht)/dvht)**2))
  n:
    inf: (1+tanh((V - vn)/dvn))/2
    tau: (tn0 + tn1 * (1 - tanh((V - vnt)/dvnt)**2))
parameters:
  C: 250 pF
  gna: 120 nS
  Ena: 50 mV
  gk: 20 nS
  Ek: -77 mV
  gl: 0.3 nS
  El: -54.4 mV
  vm: -40 mV
  dvm: 15 mV
  tm0: 0.1 ms
  tm1: 0.4 ms
  vmt: -40 mV
  dvmt: 15 mV
  vh: -60 mV
  dvh: -15 mV
  th0: 1 ms
  th1: 7 ms
  vht: -60 mV
  dvht: -15 mV
  vn: -55 mV
  dvn: 40 mV
  tn0: 1 ms
  tn1: 5 ms
  vnt: -55 mV
  dvnt: -30 mV
"""

Normally we'd use `spykscc` to build the extension module and then just import it, but for illustration purposes we'll build the module from the string descriptor.

In [6]:
from spyks import validate, build

# parse the model
pymodel = spk.load_model(doc)
# validate units
validate.check_symbols(pymodel)
validate.check_equations(pymodel)
# generate the C++ code
build_dir = os.curdir
cppfile = os.path.join(build_dir, pymodel["name"] + ".cpp")
build.write_cppfile(pymodel, cppfile)
build.build_module(cppfile, pymodel["name"], build_dir, version=pymodel["version"])

running build_ext
creating var
creating var/folders
creating var/folders/gw
creating var/folders/gw/mrwrh00d50n1p1mp95935s6w0000gn
creating var/folders/gw/mrwrh00d50n1p1mp95935s6w0000gn/T
/usr/bin/clang -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -pipe -Os -I/Users/dmeliza/.virtualenvs/akit/include -I/opt/local/Library/Frameworks/Python.framework/Versions/3.5/include/python3.5m -c /var/folders/gw/mrwrh00d50n1p1mp95935s6w0000gn/T/tmpv2ptrs3q.cpp -o var/folders/gw/mrwrh00d50n1p1mp95935s6w0000gn/T/tmpv2ptrs3q.o -std=c++14
/usr/bin/clang -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -pipe -Os -I/Users/dmeliza/.virtualenvs/akit/include -I/opt/local/Library/Frameworks/Python.framework/Versions/3.5/include/python3.5m -c /var/folders/gw/mrwrh00d50n1p1mp95935s6w0000gn/T/tmphak62npf.cpp -o var/folders/gw/mrwrh00d50n1p1mp95935s6w0000gn/T/tmp

In [7]:
# load the module
nakl = spk.load_module(pymodel, build_dir)

In [ ]:
# 